# Automated ML

Setup all imports for the notebook


In [1]:
import urllib.request
import json

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import AutoMLStep
from azureml.core import Model, Webservice
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.widgets import RunDetails

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.50.0


## Dataset

### Overview

In this exercise, I'll be using the Maternal Health Risk data set from the [UCI Machine Learning repository](https://archive.ics.uci.edu/ml/datasets/Maternal+Health+Risk+Data+Set). This data set contains 1,014 records with 7 attributes including:

* Age
* Systolic Blood Pressure as SystolicBP
* Diastolic BP as DiastolicBP
* Blood Sugar as BS
* Body Temperature as BodyTemp, 
* HeartRate
* RiskLevel
 
RiskLevel is one of 

* "low risk"
* "mid risk" 
* "high risk"

The ML model will be trained to predict the *RiskLevel* column based on the other parameters.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'mom-health'
project_folder = './pipeline-project'

experiment=Experiment(ws, experiment_name)

# Try to load the dataset from the Workspace. Otherwise, create it from the file
found = False
key = "MaternalHealth Dataset"
description_text = "Maternal Health Risk from UCI ML repository"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
        print("Found dataset already registered")

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00639/Maternal%20Health%20Risk%20Data%20Set.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)
        print("Did not find dataset -- registered")



Found dataset already registered


## Create the Compute Cluster

In [3]:
cluster_name = "mom-health-cluster"
compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", min_nodes=0, max_nodes=4)

try:
    my_compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    my_compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    my_compute_target.wait_for_completion(show_output=True)

Found existing compute target.


## AutoML Configuration

The following AutoML settings were used in this experiment:

* timeout setting of 20 minutes so that the experiment does not continue excessively
* concurrent iterations of 5 so that the nodes are efficiently used
* primary metric to optimize is accuracy as a measure of model performance 

In [4]:
automl_settings = {    
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

automl_config = AutoMLConfig(compute_target=my_compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="RiskLevel",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [5]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)
print(remote_run)


Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
mom-health,AutoML_5149cf78-e466-4c0a-8b49-56a31e874ace,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


Run(Experiment: mom-health,
Id: AutoML_5149cf78-e466-4c0a-8b49-56a31e874ace,
Type: automl,
Status: NotStarted)


## Run Details

AutoML evaluated several different models during the experiment including LightGBM, XGBoostClassifier, Random Forest, Logistic Regression, Voting Ensemble and various Forest algorithms. Overall the Voting Ensemble consistently produced the best results based on the 'accuracy' metric for which it was optimizing.

While each of the other algorithm produced results ranging from ~60%+ accuracy, the combination of the various algorithm in the Voting Ensemble gave the overall accuracy a boost with the different weighting of the models.


In [6]:
RunDetails(remote_run).show()
remote_run.wait_for_completion()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_5149cf78-e466-4c0a-8b49-56a31e874ace',
 'target': 'mom-health-cluster',
 'status': 'Completed',
 'startTimeUtc': '2023-05-01T20:34:03.262542Z',
 'endTimeUtc': '2023-05-01T20:42:25.777501Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'mom-health-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"mom-health","subscription_id":"35976fdd-d799-46b2-9fdd-859ceba2a824","resource_group":"aml-rg","workspace_name":"ud3","region":"eastus","compute_target":"mom-health-cluster","spark_service":null,"azure

## Best Model Registeratioin

Get the best model from the automl experiments, display all the properties of the model and register it.



In [7]:
# Get your best run and save the model from that run.

best_run = remote_run.get_best_child(metric='accuracy')

print(best_run.get_details())
print(best_run.get_file_names())

{'runId': 'AutoML_5149cf78-e466-4c0a-8b49-56a31e874ace_38', 'target': 'mom-health-cluster', 'status': 'Completed', 'startTimeUtc': '2023-05-01T20:41:13.160327Z', 'endTimeUtc': '2023-05-01T20:42:13.533674Z', 'services': {}, 'properties': {'runTemplate': 'automl_child', 'pipeline_id': '__AutoML_Ensemble__', 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'mom-health\',\'compute_target\':\'mom-health-cluster\',\'subscription_id\':\'35976fdd-d799-46b2-9fdd-859ceba2a824\',\'region\':\'eastus\',\'spark_service\':None}","ensemble_run_id":"AutoML_5149cf78-e466-4c0a-8b49-56a31e874ace_38","experiment_name":"mom-health","workspace_name":"ud3","subscription_id":"35976fdd-d799-46b2-9fdd-859ceba2a824","

In [8]:
best_model = best_run.register_model(model_name='automl-momhealth', model_path='outputs/model.pkl')
print(best_model.name, best_model.description, best_model.version)

best_run.download_file("outputs/scoring_file_v_1_0_0.py", "score.py")

automl-momhealth None 1


## Model Deployment

*Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.*

I chose to deploy this models created by this experiment. The following code cells deploy the model and test the endpoint with an API call.

In [9]:
# Define the inference configuration
inference_config = InferenceConfig(
    entry_script='score.py', 
    source_directory='./')

# Define the deployment configuration
deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=2, 
    memory_gb=2, 
    auth_enabled=True, 
    description="mom health API")

# Deploy the web service
service_name = 'api-momhealth'
service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[best_model],
                       overwrite=True,
                       inference_config=inference_config,
                       deployment_config=deployment_config)
service.wait_for_deployment(show_output=True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-05-01 20:46:21+00:00 Creating Container Registry if not exists.
2023-05-01 20:46:22+00:00 Use the existing image.
2023-05-01 20:46:23+00:00 Submitting deployment to compute.
2023-05-01 20:46:30+00:00 Checking the status of deployment api-momhealth..
2023-05-01 20:48:47+00:00 Checking the status of inference endpoint api-momhealth.
Succeeded
ACI service creation operation finished, operation "Succeeded"


## Test the webservice

Utilizing any of the sample data below, we can invoke the API endpoint and assess the result

In [11]:
'''
Sample data:

25,130,80,15,98,86,high risk
35,140,90,13,98,70,high risk
29,90,70,8,100,80,high risk
30,140,85,7,98,70,high risk
35,120,60,6.1,98,76,low risk
23,140,80,7.01,98,70,high risk
23,130,70,7.01,98,78,mid risk
35,85,60,11,102,86,high risk
32,120,90,6.9,98,70,mid risk
'''

# Define the input data as a JSON object
data =  {
  "data": [
    {
      "Age": 25,
      "SystolicBP": 130,
      "DiastolicBP": 80,
      "BS": 15,
      "BodyTemp": 98.0,
      "HeartRate": 86
    }
  ],
  "method": "predict"
}

body = str.encode(json.dumps(data))

headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ service.get_keys()[0])}

req = urllib.request.Request(service.scoring_uri, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))


b'"{\\"result\\": [\\"high risk\\"]}"'


## Clean up

Print the logs of the web service, delete the service, and deprovision the compute cluster.

In [12]:
print(service.get_logs())

service.delete()

try:
    my_compute_target.delete()
    my_compute_target.wait_for_completion(show_output=True)
except ComputeTargetException:
    print('ComputeTarget not found')

2023-05-01T20:48:27,685242500+00:00 - rsyslog/run 
2023-05-01T20:48:27,685338200+00:00 - gunicorn/run 
2023-05-01T20:48:27,698759700+00:00 | gunicorn/run | 
2023-05-01T20:48:27,701116600+00:00 | gunicorn/run | ###############################################
2023-05-01T20:48:27,703972000+00:00 | gunicorn/run | AzureML Container Runtime Information
2023-05-01T20:48:27,708077500+00:00 | gunicorn/run | ###############################################
2023-05-01T20:48:27,710194600+00:00 | gunicorn/run | 
2023-05-01T20:48:27,734571500+00:00 | gunicorn/run | 
2023-05-01T20:48:27,774716800+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20230412.v4
2023-05-01T20:48:27,776632600+00:00 - nginx/run 
2023-05-01T20:48:27,784222700+00:00 | gunicorn/run | 
2023-05-01T20:48:27,786715200+00:00 | gunicorn/run | 
2023-05-01T20:48:27,790734100+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/usr/local/sbi

InProgress..Current provisioning state of AmlCompute is "Deleting"

........Current provisioning state of AmlCompute is "Deleting"

........Current provisioning state of AmlCompute is "Deleting"

........Current provisioning state of AmlCompute is "Deleting"

........Current provisioning state of AmlCompute is "Deleting"

........Current provisioning state of AmlCompute is "Deleting"

........Current provisioning state of AmlCompute is "Deleting"

....
SucceededProvisioning operation finished, operation "Succeeded"
ComputeTarget not found
